## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [1]:
# load in packages
from test_results import test_results, score

# base packages
import numpy as np
import pandas as pd

# classifier libraries
from sklearn.linear_model import LogisticRegression

# library for estimator base class
from sklearn.base import BaseEstimator

# pipeline library
from sklearn.pipeline import Pipeline

from sklearn import utils

# hyperparameter search library
from sklearn.model_selection import GridSearchCV

# scoring libraries
from sklearn.metrics import make_scorer, f1_score, classification_report

%matplotlib inline


# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head(10)

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044331,-0.385883,1,1,2,2
5,9,No,0,1,41.189415,-0.905350,2,4,4,1
6,11,No,0,1,40.690409,1.085939,2,2,1,2
7,14,No,0,2,25.971529,-1.424817,1,3,3,2
8,15,Yes,0,2,31.930423,0.393317,2,3,1,2
9,16,No,0,1,27.474650,0.566472,2,2,3,2


In [2]:
# create copy of training data and clean ID
train_data2=train_data.copy()
train_data2.drop(columns=["ID"],inplace=True)  # Kick out ID


In order to determine which person is to give a promotion, one cannot simply use V1-V7 as feature variables and the promotion as target variable for training an estimator, since the promotion was given arbitarily. However, there is the information whether the person bought the product or not.

The whole training data set can thus be split according to the promoted and bought attributes leading to four subsets:
<table>
<tr><th></th><th colspan = '2'>Promotion</th></tr>
<tr><th>Buyer</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

for which we term the underlying people as follows:

* *I* - promoted buyer
* *II* - unpromoted buyer
* *III* - promoted not buyer
* *IV* - unpromoted not buyer

If we can determine people who form the intersection of *I* and *IV*, we obtain those who buy after being promoted; otherwise not.
The following Venn diagram visualizes these set of people among all four subsets *I*--*IV* under the assumption that all people's behavior is deterministic with respect to their attributes V1-V7.


<img src="venn_diagram.png" width="600" height="400">

In order to build an estimator for the black striped area forming *I^IV*, we establish at first two sub estimators trained ideally with sets whose people reveal disjunct behavior. The first trains *II* vs *IV* and the second *I* vs *III*.
Given a new person's V1-V7 attributes then the outcome of the two estimators can clearly determine whether this person is a "buyer only if promoted" or not.

However, we also have to consider that the people types are never clearly determined through their attributes or people have an unpredicatable behavior. Depending on the metrics it may often be better to rather give a promotion to a person who would buy the product anyway (gray striped area, *I* and *II*) than risking not to promote a person who is a "buyer only if promoted" (black stroüed area, *I* and *IV*).
On the other hand, if the promotion is quite expensive, you do not want to risk, that a person won't buy anyway (yellow striped area) or may be even repelled from buying when receiving it (orange striped area).

A way to quantify uncertainties is to make the estimators to predict probabilities.

In [3]:



class own_classifier(BaseEstimator):
    
    def __init__(self, prob1_threshold=0.5,prob2_threshold=0.5,prob3_threshold=0.5, estimator=[LogisticRegression(),LogisticRegression(),LogisticRegression()]):
        """constructor function"""
        self.set_params(prob1_threshold,prob2_threshold,prob3_threshold, estimator)
        return None

    def set_params(self,prob1_threshold,prob2_threshold,prob3_threshold, estimator):
        """parameters are set"""
        self.estimator = estimator
        self.prob1_threshold=prob1_threshold
        self.prob2_threshold=prob2_threshold
        self.prob3_threshold=prob3_threshold
        

        return self
        
    def combine_probabilities_into_output(self,prob1,prob2,prob3):
        """
        It is aimed to combine two preditor probabilities into one
        
        INPUT
        prob1 - probability of event 1
        prob2 - probability of event 2
        
        OUTPUT
        "YES" or "NO" depending on the combined condition
        """

        if (  (prob1>self.prob1_threshold) and (prob2>self.prob2_threshold ) and (prob3>self.prob3_threshold)  ) :
            return "Yes"
        else:
            return "No"
        
        return None
    
    def own_metrics(self,df, promo_pred_col = 'Promotion'):

        n_treat       = df.loc[df[promo_pred_col] == 'Yes',:].shape[0]
        n_control     = df.loc[df[promo_pred_col] == 'No',:].shape[0]
        n_treat_purch = df.loc[df[promo_pred_col] == 'Yes', 'purchase'].sum()
        n_ctrl_purch  = df.loc[df[promo_pred_col] == 'No', 'purchase'].sum()
        irr = n_treat_purch / n_treat - n_ctrl_purch / n_control
        nir = 10 * n_treat_purch - 0.15 * n_treat - 10 * n_ctrl_purch
        # if (n_treat==0):
        #     return (-10000,-1000000)
        # if (n_control==0):
        #     return (-10000,-1000000)
        return (irr, nir)

    
    def score(self, X, y):
       """scoring function"""   
       y_pred=self.predict(X[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']])
       score_df=X.iloc[np.where(y_pred=="Yes")]
       irr, nir = self.own_metrics(score_df)

       return abs(irr)*abs(nir)*(np.sign(irr)+np.sign(nir))

    def fit(self,X,y=None):
        """fit procedure covering all classifiers for each target"""

        # build subsets
        train_data_sure_buyer=(X[(X.purchase==1) & (X.Promotion=="No")]).copy()
        train_data_unpromoted_not_buyer=(X[(X.purchase==0) & (X.Promotion=="No")]).copy()
        train_data_sure_not_buyer=(X[(X.purchase==0) & (X.Promotion=="Yes")]).copy()
        train_data_promoted_buyer=(X[(X.purchase==1) & (X.Promotion=="Yes")]).copy()
        


        # adjust dataframes
        train_data_sure_buyer.drop(columns=['Promotion','purchase'],inplace=True)
        train_data_sure_buyer['parameter']=0

        train_data_unpromoted_not_buyer.drop(columns=['Promotion','purchase'],inplace=True)
        train_data_unpromoted_not_buyer['parameter']=1

        train_data_sure_not_buyer.drop(columns=['Promotion','purchase'],inplace=True)
        train_data_sure_not_buyer['parameter']=0

        train_data_promoted_buyer.drop(columns=['Promotion','purchase'],inplace=True)
        train_data_promoted_buyer['parameter']=1

        
        # create 2 more subsets in which those who do not need a promotion have a value of zero and those who need or perhaps (we do not know) may not need a promotion get value 1.
        temp1=pd.concat([train_data_sure_buyer, train_data_unpromoted_not_buyer])
        temp2=pd.concat([train_data_sure_not_buyer, train_data_promoted_buyer])


        temp3=pd.concat([train_data_sure_buyer,train_data_sure_not_buyer, train_data_unpromoted_not_buyer,train_data_promoted_buyer])

        
        # build train and test set for train_data_sure_buyer_and_unpromoted_not_buyer
        X_train=temp1[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']]
        y_train=np.array(temp1[['parameter']].values).ravel()

        # build train and test set for train_data_sure_not_buyer_and_promoted_buyer
        X_train2=temp2[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']]
        y_train2=np.array(temp2[['parameter']].values).ravel()
        
        X_train3=temp3[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']]
        y_train3=np.array(temp3[['parameter']].values).ravel()
        
        # fit (sub) estimators
        self.estimator[0].fit(X_train,y_train)
        self.estimator[1].fit(X_train2,y_train2)
        self.estimator[2].fit(X_train3,y_train3)


       

        return self

    def predict(self, X):
        """predict-funtion for the only one target value"""

        y_pred1 = self.estimator[0].predict_proba(X.iloc[:,:]).T[1]
        y_pred2 = self.estimator[1].predict_proba(X.iloc[:,:]).T[1]
        y_pred3 = self.estimator[2].predict_proba(X.iloc[:,:]).T[1]
        
        return np.array(list(map(self.combine_probabilities_into_output,y_pred1,y_pred2,y_pred3)))

In [4]:
def build_model():
    """Definition of the model via pipeline and a parameters variable.
    The best parameters were determined using grid search.
    """

    pipeline= Pipeline([
        ('clf',own_classifier())
    ])


    parameters = [
        {
            "clf__estimator": ([
                [LogisticRegression(random_state=0, C=1, max_iter=5000, class_weight="balanced" ),LogisticRegression(random_state=0, C=1, max_iter=5000, class_weight="balanced" ),LogisticRegression(random_state=0, C=1, max_iter=5000, class_weight="balanced" )]
            ]),
            # "clf__prob1_threshold": ( list(np.arange(0.4,0.7,0.02))),
            # "clf__prob2_threshold": ( list(np.arange(0.4,0.7,0.02))),
            # "clf__prob3_threshold": ([0,0.495,0.5,0.505]),
            "clf__prob1_threshold": ([0.46]),
            "clf__prob2_threshold": ([0.56]),
            "clf__prob3_threshold": ([0.5]),
        }
    ]

    cv = GridSearchCV(
        pipeline,
        param_grid=parameters,
        refit=True,
        cv=8,
        n_jobs=-1
    )
    return cv
    

In [5]:
# build and fit model
cv=build_model()
cv.fit(train_data2)


GridSearchCV(cv=8, estimator=Pipeline(steps=[('clf', own_classifier())]),
             n_jobs=-1,
             param_grid=[{'clf__estimator': [[LogisticRegression(C=1,
                                                                 class_weight='balanced',
                                                                 max_iter=5000,
                                                                 random_state=0),
                                              LogisticRegression(C=1,
                                                                 class_weight='balanced',
                                                                 max_iter=5000,
                                                                 random_state=0),
                                              LogisticRegression(C=1,
                                                                 class_weight='balanced',
                                                                 max_iter=5000,
                                                                 random_state=0)]],
                          'clf__prob1_threshold': [0.46],
                          'clf__prob2_threshold': [0.56],
                          'clf__prob3_threshold': [0.5]}])

In [6]:
def promotion_strategy(df):
    '''
    INPUT
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an
                   individual should recieve a promotion
                   should be the length of df.shape[0]

    Ex:
    INPUT: df

    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2

    OUTPUT: promotion

    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and
    the last should not.
    '''

    promotion_df=cv.predict(df)

    return promotion_df

In [7]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0190.

Your nir with this strategy is 250.45.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.01896753159237121, 250.45000000000005)